## Test against onnx model that is saved locally

If Success, repeat format against model hosted through API

In [17]:
import onnxruntime as ort
import numpy as np
import pandas as pd
import joblib

# Load the training columns (make sure 'training_columns.pkl' exists in the directory)
training_columns = joblib.load("training_columns.pkl")

# Define a sample input for testings
new_data_default_risk = {
    'AMT_INCOME_TOTAL': 50000,
    'AMT_CREDIT': 600000,
    'AMT_ANNUITY': 25000,
    'AMT_GOODS_PRICE': 550000,
    'DAYS_BIRTH': -16000,
    'DAYS_EMPLOYED': 0,
    'REGION_POPULATION_RELATIVE': 0.04,
    'CNT_FAM_MEMBERS': 4,
    'FLAG_MOBIL': 1,
    'FLAG_EMAIL': 1,
    'FLAG_WORK_PHONE': 0,
    'NAME_INCOME_TYPE_Working': 0,
    'NAME_INCOME_TYPE_Unemployed': 1,
    'NAME_EDUCATION_TYPE_Higher_education': 0,
    'NAME_EDUCATION_TYPE_Secondary_education': 1,
    'NAME_FAMILY_STATUS_Married': 0,
    'NAME_FAMILY_STATUS_Single': 1,
    'NAME_HOUSING_TYPE_House_apartment': 1,
    'NAME_HOUSING_TYPE_With_parents': 0,
    'OCCUPATION_TYPE_Laborers': 1,
    'OCCUPATION_TYPE_Sales_staff': 0
}

# Convert the dictionary to a DataFrame and reindex to match the training columns
test_data = pd.DataFrame([new_data_default_risk])
test_data = test_data.reindex(columns=training_columns, fill_value=0)

# Load your saved preprocessor
best_model = joblib.load("best_model.joblib")
preprocessor = best_model.named_steps['preprocessor']

# Preprocess the test data
test_data_processed = preprocessor.transform(test_data).astype(np.float32)

# Load the ONNX model
onnx_model_path = "models/1/xgb_classifier.onnx"  # Ensure this file path is correct
session = ort.InferenceSession(onnx_model_path)

# Prepare input data in the required format for ONNX (numpy array)
input_name = session.get_inputs()[0].name
test_data_array = test_data_processed

# Run the model and get the output
prediction = session.run(None, {input_name: test_data_array})[0]

# Display the prediction result
predicted_class = "Default" if prediction[0] == 1 else "Non-default"
print(f"Predicted Class: {predicted_class}")
print(f"Prediction Probability: {prediction}")


Predicted Class: Non-default
Prediction Probability: [0]


## Test against model hosted through API

In [15]:
import pandas as pd
import numpy as np
import joblib
import requests
import json

# deployed_model_name = "microfinanceloan"
# infer_endpoint = "http://modelmesh-serving.demo:8008"
# infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

# infer_url = "http://modelmesh-serving.demo.svc.cluster.local:8008/v2/models/microfinanceloan/infer"
infer_url = "https://xgb.demo.svc.cluster.local/v2/models/xgb/infer"
# https://xgb.demo.svc.cluster.local
# http://xgb-predictor.demo.svc.cluster.local

# Load the training columns and preprocessor
training_columns = joblib.load("training_columns.pkl")
best_model = joblib.load("best_model.joblib")
preprocessor = best_model.named_steps['preprocessor']

# Define the sample input
new_data_default_risk = {
    'AMT_INCOME_TOTAL': 50000,
    'AMT_CREDIT': 600000,
    'AMT_ANNUITY': 25000,
    'AMT_GOODS_PRICE': 550000,
    'DAYS_BIRTH': -16000,
    'DAYS_EMPLOYED': 0,
    'REGION_POPULATION_RELATIVE': 0.04,
    'EXT_SOURCE_1': 0.1,
    'EXT_SOURCE_2': 0.15,
    'EXT_SOURCE_3': 0.2,
    'CNT_FAM_MEMBERS': 4,
    'FLAG_MOBIL': 1,
    'FLAG_EMAIL': 1,
    'FLAG_WORK_PHONE': 0,
    'NAME_INCOME_TYPE_Working': 0,
    'NAME_INCOME_TYPE_Unemployed': 1,
    'NAME_EDUCATION_TYPE_Higher_education': 0,
    'NAME_EDUCATION_TYPE_Secondary_education': 1,
    'NAME_FAMILY_STATUS_Married': 0,
    'NAME_FAMILY_STATUS_Single': 1,
    'NAME_HOUSING_TYPE_House_apartment': 1,
    'NAME_HOUSING_TYPE_With_parents': 0,
    'OCCUPATION_TYPE_Laborers': 1,
    'OCCUPATION_TYPE_Sales_staff': 0
}

# Convert the dictionary to a DataFrame and reindex to match the training columns
test_data = pd.DataFrame([new_data_default_risk])
test_data = test_data.reindex(columns=training_columns, fill_value=0)

# Preprocess the test data
test_data_processed = preprocessor.transform(test_data).astype(np.float32)

# Flatten the processed data and ensure it’s in a list format
data_list = test_data_processed.flatten().tolist()

# Define payload with explicit shape, datatype, and the processed data
payload = {
    "inputs": [
        {
            "name": "input_0",  # Use the expected input name if different
            "shape": [1, len(data_list)],  # Adjust shape to match your data
            "datatype": "FP32",
            "data": data_list
        }
    ]
}

# Define API endpoint and headers
headers = {"Content-Type": "application/json"}

# Send POST request
response = requests.post(infer_url, headers=headers, data=json.dumps(payload), verify=False)

# Check response
if response.status_code == 200:
    try:
        # Extract predictions
        prediction = response.json().get("outputs", [{}])[0].get("data")
        predicted_class = "Default" if prediction[0] == 1 else "Non-default"
        print(f"Predicted Class: {predicted_class}")
        print(f"Prediction Probability: {prediction}")
    except Exception as e:
        print("Error parsing response:", e)
        print("Response content:", response.json())
else:
    print(f"Request failed with status code {response.status_code}")
    print("Response:", response.text)


Request failed with status code 404
Response: {"error": "Model with requested version is not found"}


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'xgb.demo.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
